In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')

In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 142kB/s  eta 0:00:01     |█████████████████████████▍      | 172.5MB 41.6MB/s eta 0:00:02
     |████████████████████████████████| 204kB 33.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

***MEAN***

In [5]:
rdd = sc.parallelize([1,2,4,5,34,1,32,4,34,2,1,3])

In [6]:
sum = rdd.sum()
count = rdd.count()
mean = sum / count
print('%.3f'%mean)

10.250


***MEDIAN***

In [53]:
ord = rdd.sortBy(lambda x : x ).zipWithIndex().map(lambda x: (x[1],x[0])).collect()
print (ord,len(ord))

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 2), (5, 3), (6, 4), (7, 4), (8, 5), (9, 32), (10, 34), (11, 34)] 12


In [54]:
n = len(ord)
if (n % 2 ==1):
    index = int((n-1)/2)
    print( ord[index][1])
else:
    index1 = int((n/2)-1)
    index2 = int((n/2))
    print( ord[index1][1])
    print( ord[index2][1])
    val1 = ord[index1][1]
    val2 = ord[index2][1]
    med = (val1 + val2)/2
    print('%.3f'%med)

3
4
3.500


***STANDARD DEVIATION***

In [72]:
#standard deviation
rdd1 = sc.parallelize([34,1,23,4,3,3,12,4,3,1])
sum = rdd1.sum()
count1 = rdd1.count()
mean1 = sum / count1
print('%.3f'%mean1)

8.800


In [73]:
from math import sqrt
print(sqrt(rdd1.map(lambda x: pow(x-mean1,2)).sum()/count1))

10.562196741208714


***KURTOSIS***

In [7]:
rdd2 = sc.parallelize([34,1,23,4,3,3,12,4,3,1])

In [9]:
#mean
sum = rdd2.sum()
count2 = rdd2.count()
mean2 = sum / count2
print('%.3f'%mean2)

8.800


In [11]:
#std
from math import sqrt
std=sqrt(rdd2.map(lambda x: pow(x-mean2,2)).sum()/count2)
print(std)

10.562196741208714


In [13]:
kt = rdd2.map(lambda x : pow(x-mean2,4)/pow(std,4)).sum()/count2
print(kt)

3.663124005193276


***Skewness***

In [14]:
sk = rdd2.map(lambda x : pow(x-mean2,3)/pow(std,3)).sum()/count2
print(sk)

1.4549069329914


In [40]:
import random
rddX = sc.parallelize(random.sample(range(100),100))
rddY = sc.parallelize(random.sample(range(100),100))

In [41]:
meanX = rddX.sum()/rddX.count()
meanY = rddY.sum()/rddY.count()
print("meanX {:.5f}".format(meanX))
print("meanY {:.5f}".format(meanY))

meanX 49.50000
meanY 49.50000


In [42]:
rddXY = rddX.zip(rddY)
rddXY.take(10)

[(27, 88),
 (39, 96),
 (62, 43),
 (35, 0),
 (80, 39),
 (23, 91),
 (63, 60),
 (52, 68),
 (59, 15),
 (47, 57)]

***COVARIANCE***

In [43]:
covXY = rddXY.map(lambda x : (x[0]-meanX)*(x[1]-meanY)).sum()/rddXY.count()
print(covXY)

54.96


In [90]:
import random
rddX = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rddY = sc.parallelize([7,6,5,4,5,6,7,8,9,10])
meanX = rddX.sum()/rddX.count()
meanY = rddY.sum()/rddY.count()
print("meanX {:.5f}".format(meanX))
print("meanY {:.5f}".format(meanY))
rddXY = rddX.zip(rddY)
rddXY.take(10)
covXY = rddXY.map(lambda x : (x[0]-meanX)*(x[1]-meanY)).sum()/rddXY.count()
print(covXY)

meanX 5.50000
meanY 6.70000
3.65


***CORRELATION***

In [44]:
n = rddXY.count()
stdX=sqrt(rddX.map(lambda x: pow(x-meanX,2)).sum()/n)
stdY=sqrt(rddX.map(lambda x: pow(x-meanY,2)).sum()/n)
corrXY = covXY/(stdX * stdY)
corrXY

0.06595859585958595

***CORRELATION MATRIX***

In [49]:
import random
col1 = sc.parallelize(range(100))
col2 = sc.parallelize(range(100,200))
col3 = sc.parallelize(list(reversed(range(100))))
col4 = sc.parallelize(random.sample(range(100),100))

In [50]:
from pyspark.mllib.stat import Statistics

In [88]:
data = col1.zip(col2).zip(col3).zip(col4).map(lambda x : (x[0][0][0],x[0][0][1],x[0][1],x[1])).map(lambda x: [x[0],x[1],x[2],x[3]])
data.take(10)

[[0, 100, 99, 61],
 [1, 101, 98, 0],
 [2, 102, 97, 64],
 [3, 103, 96, 16],
 [4, 104, 95, 86],
 [5, 105, 94, 15],
 [6, 106, 93, 66],
 [7, 107, 92, 91],
 [8, 108, 91, 19],
 [9, 109, 90, 46]]

In [89]:
Statistics.corr(data)

array([[ 1.        ,  1.        , -1.        , -0.01306931],
       [ 1.        ,  1.        , -1.        , -0.01306931],
       [-1.        , -1.        ,  1.        ,  0.01306931],
       [-0.01306931, -0.01306931,  0.01306931,  1.        ]])

***QUIZ***

In [95]:
import random
rddX = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rddY = sc.parallelize([7,6,5,4,5,6,7,8,9,10])
meanX = rddX.sum()/rddX.count()
meanY = rddY.sum()/rddY.count()
print("meanX {:.5f}".format(meanX))
print("meanY {:.5f}".format(meanY))
rddXY = rddX.zip(rddY)
covXY = rddXY.map(lambda x : (x[0]-meanX)*(x[1]-meanY)).sum()/rddXY.count()
print(covXY)

meanX 5.50000
meanY 6.70000
3.65


In [98]:
n = rddXY.count()
stdX=sqrt(rddX.map(lambda x: pow(x-meanX,2)).sum()/n)
stdY=sqrt(rddX.map(lambda x: pow(x-meanY,2)).sum()/n)
corrXY = covXY/(stdX * stdY)
print(corrXY)

0.40822913961171386
